In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018', 'Winterthur 2017', 'Winterthur 2018', 'Winterthur 2019', 'Winterthur 2020', 'USZ 2018', 'USZ 2019', 'USZ 2019_30_04_2020', 'KSSG 2021', 'KSSG Linth 2019', 'KSSG Toggenburg 2019', 'KSSG SRRWS 2019', 'Linth_Toggenburg_SRRWS_2019.xlsx'])

In [3]:
file = FILES_TO_ANALYZE['KSSG Linth 2019']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Linth_Toggenburg_SRRWS_2019.xlsx', hospital_name_db='KSSG Linth', year='2019', sheets=['Änderungen_Spital_Linth_2019'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 15 cases for KSSG Linth 2019
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [6]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,3108134918,DDB3BDB39DE3F695,M,49,7,3,J64C,A46,M100,K746,<NA>,<NA>,<NA>
1,3108136936,1C6A9D7FAA9A7548,W,89,5,2,G46C,K226,M100,N184,D6833,<NA>,<NA>
2,3108141419,69231F9C61534D42,M,69,10,2,G60B,C183,M100,E43,<NA>,<NA>,<NA>
3,3150158985,29F13DDB90E1976D,W,70,2,3,G48C,K922,M100,"D62,D684",D500,<NA>,<NA>
4,3150164388,11721C6EB40D5BB5,M,78,5,2,G46C,K250,M100,D62,<NA>,<NA>,<NA>


# match to the database


In [7]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['3108134918', '3108136936', '3108141419', '3150158985',
       '3150164388', '3108135794', '3150163534', '3108132452',
       '3108136823', '3108139676', '3150154963', '3150163156',
       '3108132184', '3108129511', '3108130840'], dtype=object)

In [8]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,0.801,249806,11,3108129511,486B44F6E09844DD,58,0,M,9,3,0,0,0,2019-01-02,01,2019-01-11,00,H63D,H63,3
1,0.693,252295,11,3108130840,5E420CB29D2534F4,72,0,W,5,3,0,0,0,2019-02-18,01,2019-02-23,00,E65C,E65,3
2,0.799,253000,11,3108132184,00B102213B2138A1,72,0,M,12,3,0,0,0,2019-03-22,01,2019-04-03,00,L60D,L60,3
3,1.565,251241,11,3108132452,DA9064B3A6941B98,84,0,M,9,4,0,0,0,2019-04-01,01,2019-04-10,00,I12C,I12,2
4,0.562,251228,11,3108134918,DDB3BDB39DE3F695,49,0,M,7,3,0,0,0,2019-05-31,01,2019-06-07,00,J64C,J64,3


In [9]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 15 out of 15 revised cases from DtoD that are matched with the database for KSSG Linth 2019


In [10]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match